In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import random

In [4]:
data = pd.read_csv('datasets/receipts_categorized.csv')
data = data.drop(columns=['Unnamed: 0'])
data

,transaction_key,item_hierarchy_sub_department,item_hierarchy_type,item_hierarchy_sub_type,line_article_id,line_article_description,line_item_price,sum_receipt,client_type,line_total_sum_after_pricecorrection_with_vat,datetime,count_of_items,category_id,category_name
0,009_002_71_2022-10-04 21:52:48_2203,305,10,30,10318303,РАМКА 1-ПОСТОВАЯ VALENA БЕЛЫЙ,63.0,4049.5,B2C,63.0,2022-10-04 21:52:48,1.0,200306.0,Накладки для включателей и розеток
1,009_002_14_2022-06-18 17:57:09_49,415,30,50,13738901,CR_НОЖ 18 ММ ДВУХКОМП. РУЧКА 1 ЛЕЗВИЕ,86.0,1516.0,B2C,86.0,2022-06-18 17:57:09,1.0,200736.0,Нож строительный
2,009_002_11_2022-10-15 09:57:34_9878,1020,10,40,82182119,ШСММ ПШ ОСТР ОЦ 4.2Х19 УП.200ШТ ST,172.0,512.0,B2C,172.0,2022-10-15 09:57:34,1.0,201994.0,Шуруп для дерева
3,009_002_18_2022-06-06 14:20:36_5879,1135,20,10,83182721,"КЛЕЙ ПВА Д/ВНУТР РАБОТ, 1КГ",226.0,1011.0,B2C,226.0,2022-06-06 14:20:36,1.0,202235.0,Столярный клей
4,009_002_9_2022-08-06 14:56:52_5444,1351,30,20,12532741,"ЛАМПА ГАЛОГЕН.UNIEL G9, 40W, 230W МАТОВ.",46.0,1807.0,B2C,460.0,2022-08-06 14:56:52,10.0,202358.0,Лампочка
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9238978,009_002_12_2022-03-07 18:02:15_5955,940,20,100,82097811,ПОДСОЛНЕЧНИК БИТЮГ 10 Г Н17,17.0,5547.2,B2C,17.0,2022-03-07 18:02:15,1.0,201278.0,"Семена овощей, пряных трав, плодово-ягодных ра..."
9238979,009_002_19_2022-08-09 21:12:11_5851,410,30,30,18800891,"CR_КУСАЧКИ ДИАГОНАЛЬНЫЕ, 160ММ, DEXTER",521.0,2561.0,B2C,521.0,2022-08-09 21:12:11,1.0,200479.0,"Кусачки, бокорезы"
9238980,009_002_14_2022-01-23 12:15:42_7432,335,50,20,18156206,ЭЛЕМЕНТ ПЛОСК КАНАЛА EQUATION 55Х110 L60,120.0,3287.7,B2C,120.0,2022-01-23 12:15:42,1.0,200295.0,Вентиляционные аксессуары
9238981,009_002_4_2022-06-02 14:36:21_8532,1020,20,50,84039098,ДЮБЕЛЬ ТЕПЛ ПЛАСТ ГВ СЕР IZO10Х90 30ШТ,126.0,1481.0,B2C,126.0,2022-06-02 14:36:21,1.0,202014.0,Дюбель для всех видов стен


In [ ]:
data_new = pd.read_csv('datasets/receipts_new.csv')
data_new

In [ ]:
grouped_by_receipts = data.groupby(['transaction_key'])
grouped_by_receipts.describe()

C:\Users\yulya\AppData\Local\Temp\ipykernel_13200\2916983998.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  receipt['product_price'] = receipt[['line_item_price', 'count_of_items']].apply(lambda x:  mul(x), axis=1)


         line_item_price                  line_article_description  \
8626159            372.0          ПОДВОДКА Д/ВОДЫ УГЛ. 1/2 ВР 60СМ   
3129290            320.0    КОЛОДКА 4 ПОСТА С ВЫКЛЮЧ. С/З, 3500 ВТ   
1579954            303.0       ОТКОС ОКОННЫЙ УТЕПЛЕН 1500Х400Х10ММ   
4085922            298.0    СВЕРХСИЛЬНАЯ МОНТАЖНАЯ ЛЕНТА 2,5X300СМ   
4410068            298.0       РОЗЕТКА С/З БЕЗ ШТОРОК VALENA БЕЛЫЙ   
7240719            175.0    ШЛАНГ СЛИВНОЙ Д/СТИР.МАШ EQUATION 2,5М   
5757018            153.0         GLADE АЭРОЗ КОКОСОВЫЙ БАЛИ 300 МЛ   
7584297            123.0  ТЖ КАСС_БАТАРЕЙКА АЛК AAA 4ШТ LEXMAN INT   
7932499            114.0         ПЕТЛЯ УНИВЕРС.100Х70Х2 МАТ.ЗОЛОТО   
2565383             92.0       ГОФРОТРУБА 1/4Х40/50 800ММ С ГАЙКОЙ   
5401212             64.0             ХОМУТ НЕРЖ. W2 8-12 ММ (2 ШТ)   
0                   63.0             РАМКА 1-ПОСТОВАЯ VALENA БЕЛЫЙ   
420332              14.2             КОРОБКА УСТ ПОЛ.СТЕН D68Х45ММ   
7005569             

C:\Users\yulya\AppData\Local\Temp\ipykernel_13200\2916983998.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  receipt['product_price'] = receipt[['line_item_price', 'count_of_items']].apply(lambda x:  mul(x), axis=1)
C:\Users\yulya\AppData\Local\Temp\ipykernel_13200\2916983998.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  receipt['product_price'] = receipt[['line_item_price', 'count_of_items']].apply(lambda x:  mul(x), axis=1)


       line_item_price            line_article_description  product_price
2                172.0  ШСММ ПШ ОСТР ОЦ 4.2Х19 УП.200ШТ ST          172.0
42331            168.0  ШСММ ПШ ОСТР ОЦ 4.2Х25 УП.150ШТ ST          168.0
         line_item_price                  line_article_description  \
2896275            433.0    CR_КЛЕЙ ГИБРИД ПОЛИМЕР AXTON,290МЛ,БЕЛ   
3                  226.0               КЛЕЙ ПВА Д/ВНУТР РАБОТ, 1КГ   
6915163            177.0  ИЗОЛЯЦИЯ K-FLEX COMPACT 22/4ММ КРАСН 10М   
8368485            175.0    ШЛАНГ СЛИВНОЙ Д/СТИР.МАШ EQUATION 2,5М   

         product_price  
2896275          433.0  
3                226.0  
6915163          177.0  
8368485          175.0  


C:\Users\yulya\AppData\Local\Temp\ipykernel_13200\2916983998.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  receipt['product_price'] = receipt[['line_item_price', 'count_of_items']].apply(lambda x:  mul(x), axis=1)
C:\Users\yulya\AppData\Local\Temp\ipykernel_13200\2916983998.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  receipt['product_price'] = receipt[['line_item_price', 'count_of_items']].apply(lambda x:  mul(x), axis=1)


...

C:\Users\yulya\AppData\Local\Temp\ipykernel_13200\2916983998.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  receipt['product_price'] = receipt[['line_item_price', 'count_of_items']].apply(lambda x:  mul(x), axis=1)
C:\Users\yulya\AppData\Local\Temp\ipykernel_13200\2916983998.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  receipt['product_price'] = receipt[['line_item_price', 'count_of_items']].apply(lambda x:  mul(x), axis=1)


         line_item_price                  line_article_description  \
7664719           3996.0               СИДЕНИЕ ДЛЯ УНИТАЗА  BAS SО   
2050672            698.0    ГЕРМЕТИК TYTAN UPG TURBO САН 280МЛ БЕЛ   
1000091            403.0            УДАЛИТЕЛЬ СИЛИКОНА AXTON 100МЛ   
5939089            313.0         УДАЛИТЕЛЬ ПЛЕСЕНИ NEOMID,ГОТ,0,5Л   
7293785            242.0    ЩЕТКА ДЛЯ ПОСУДЫ РЕЗИНОВАЯ YOU LL LOVE   
6984392            236.0       CR_ПИСТОЛЕТ СКЕЛЕТНЫЙ МЕТАЛЛИЧЕСКИЙ   
3178078            192.0         ТУАЛ.УТЁНОК ВИДИМЫЙ ЭФФЕКТ 900 МЛ   
6                  181.0         CR_САЛФЕТКА МИКРОФИБРА 4ШТ. 35/35   
6270720            160.0  СПРЕЙ ДЛЯ ДУШЕВЫХ КАБИН И АКРИЛОВЫХ ВАНН   
1709232             70.0  НАБОР ГУБОК ДЛЯ ПОСУДЫ 5Х65Х43ММ 5 ШТ/УП   

         product_price  
7664719         3996.0  
2050672          698.0  
1000091          403.0  
5939089          313.0  
7293785          242.0  
6984392          236.0  
3178078          192.0  
6                181.0  
6270

C:\Users\yulya\AppData\Local\Temp\ipykernel_13200\2916983998.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  receipt['product_price'] = receipt[['line_item_price', 'count_of_items']].apply(lambda x:  mul(x), axis=1)
C:\Users\yulya\AppData\Local\Temp\ipykernel_13200\2916983998.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  receipt['product_price'] = receipt[['line_item_price', 'count_of_items']].apply(lambda x:  mul(x), axis=1)


         line_item_price                 line_article_description  \
1540746           4771.0                 ДПГ КАНТРИ ХВОЯ 200Х80СМ   
1209499            868.0  РУЧКА-ЗАЩЁЛКА КЛЮЧ/ФИКС. ЗОЛОТО 8023-01   
2371610            371.0   БРУС КОРОБОЧНЫЙ ХВОЯ СРАЩ А 40X65Х2100   
8022896            203.0   НАЛИЧНИК ХВОЯ СРАЩ РАДИУС А 10Х65Х2200   
8                  170.0          ЖИРОУДАЛИТЕЛЬ AZELIT КАЗАН 0,6Л   
7613913            116.0     ПЕТЛЯ Б/ВРЕЗКИ 100Х75Х2,5 МАТ.ЗОЛОТО   
5768266             85.0         ПЕТЛЯ Б/ВРЕЗКИ 100Х75Х2,5 БРОНЗА   

         product_price  
1540746         4771.0  
1209499          868.0  
2371610         1855.0  
8022896         1015.0  
8                170.0  
7613913          116.0  
5768266           85.0  
         line_item_price                  line_article_description  \
6258890            913.0       ГЛАЗ.КЕР VILIO КОР.РЕЛЬЕФ 29,8X29,8   
4023044            436.0     КЛЕЙ Д/ПЛИТКИ С1T PLITONIT В PRO 25КГ   
6513157            182.0         ЗАТИ

C:\Users\yulya\AppData\Local\Temp\ipykernel_13200\2916983998.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  receipt['product_price'] = receipt[['line_item_price', 'count_of_items']].apply(lambda x:  mul(x), axis=1)


In [2]:
data_new = pd.read_csv('receipts_new.csv')


,gid,transaction_key,store_id,opened_date,line_item_id,line_quantity,line_item_price,line_item_cost,line_margin,line_type
0,BipqVCA9TxmTukbRjRKJXg,009_002_74_2022-04-18 12:22:18_9058,2,2022-04-18,83591944,1.0,2990.0,2516.10,-24.43,Sales
1,b0R_ll-dSsSbcMvCgNhqyA,009_003_35_2022-08-26 18:00:29_7374,3,2022-08-26,15058563,2.0,1413.0,1206.73,412.54,Undefined
2,b0R_ll-dSsSbcMvCgNhqyA,009_003_35_2022-08-26 18:00:29_7374,3,2022-08-26,82148968,1.0,338.0,179.51,158.49,Undefined
3,b0R_ll-dSsSbcMvCgNhqyA,009_003_35_2022-08-26 18:00:29_7374,3,2022-08-26,82148968,3.0,338.0,179.51,475.47,Undefined
4,b0R_ll-dSsSbcMvCgNhqyA,009_003_35_2022-08-26 18:00:29_7374,3,2022-08-26,16042299,1.0,1634.0,1402.83,231.17,Undefined
...,...,...,...,...,...,...,...,...,...,...
27601037,DJf64XaDQSWhZgwwXld0nw,009_002_75_2022-09-08 11:14:12_4196,2,2022-09-08,12240853,21.0,43.0,44.66,-185.36,Sales
27601038,DJf64XaDQSWhZgwwXld0nw,009_002_75_2022-09-08 11:14:12_4196,2,2022-09-08,12240853,1.0,43.0,44.66,-8.83,Sales
27601039,DJf64XaDQSWhZgwwXld0nw,009_002_75_2022-09-08 11:14:12_4196,2,2022-09-08,82258971,1.0,950.0,969.86,-178.19,Sales
27601040,DJf64XaDQSWhZgwwXld0nw,009_002_75_2022-09-08 11:14:12_4196,2,2022-09-08,49000003,1.0,4.0,NaN,NaN,Sales


In [28]:
# dupl = data_new.duplicated()
np.where(dupl == True)[0]

[      16       17       20 ... 27601013 27601015 27601034]


In [17]:
def mul(x):
    m = 1.0
    for i in x:
        m *= i
    return m

dict_product_pos = {}
# Товар: [позиция товара в чеках]

dict_product_freq = {}
# Товар: счетчик появления товара в чеках

def set_pos(x, array_of_prices):
    item = x['line_item_id']
    price = x['product_price']
    if  item not in dict_product_pos:
        dict_product_pos[item] = []
    dict_product_pos[item].append(np.where(array_of_prices == price)[0][0])
    
        
def analyze_receipt(receipt):
    receipt[['line_item_price', 'line_quantity']]
    product_prices = receipt[['line_item_price', 'line_quantity']].apply(lambda x: mul(x), axis=1)
    receipt['product_prices'] = product_prices
#     print(receipt['product_price'])
    sorted_receipt = receipt.sort_values(['product_price'], ascending=False)
    array_of_prices = sorted_receipt['product_price']
    sorted_receipt.apply(lambda x: set_pos(x, array_of_prices), axis=1)
    
#     print(np.where(array_of_prices == ))

keys = data_new['transaction_key'].unique()
for key in keys[:5]:
    receipt = data_new.loc[data_new['transaction_key'] == key]
    analyze_receipt(receipt)

C:\Users\yulya\AppData\Local\Temp\ipykernel_12876\4024586353.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  receipt['product_price'] = receipt[['line_item_price', 'line_quantity']].apply(lambda x: mul(x), axis=1)
C:\Users\yulya\AppData\Local\Temp\ipykernel_12876\4024586353.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  receipt['product_price'] = receipt[['line_item_price', 'line_quantity']].apply(lambda x: mul(x), axis=1)
C:\Users\yulya\AppData\Local\Temp\ipykernel_12876\4024586353.py:23: Settin

In [83]:
# dict_product_pos
# np.where(data_new['transaction_key'] in keys[:5])
# keys_group = data_new.groupby(['transaction_key'])
# data_new.loc[data_new['line_item_id'] == 82148960]

data_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27601042 entries, 0 to 27601041
Data columns (total 10 columns):
 #   Column           Dtype  
---  ------           -----  
 0   gid              object 
 1   transaction_key  object 
 2   store_id         int64  
 3   opened_date      object 
 4   line_item_id     int64  
 5   line_quantity    float64
 6   line_item_price  float64
 7   line_item_cost   float64
 8   line_margin      float64
 9   line_type        object 
dtypes: float64(4), int64(2), object(4)
memory usage: 2.1+ GB


In [ ]:
# cnt = 10
for key in keys_group.groups:
    cnt -= 1
    lines = keys_group.groups[key].array
    receipt = data_new.iloc[lines]
    dupl_receipt = receipt.duplicated(subset=['line_item_id'], keep=False)
#     print(receipt)
    duplicates_indices = dupl_receipt.loc[dupl_receipt == True].index.to_numpy()
    if len(duplicates_indices) != 0:
        data_new.at[duplicates_indices[0], 'line_quantity'] += data_new.loc[duplicates_indices[1:], 'line_quantity'].sum()
        data_new = data_new.drop(duplicates_indices[1:])
        

data_new.info()
    
#     dupl_indices = np.where(dupl_receipt == True)[0]
    
#     if cnt == 0:
#         break
    